# Home Credit Modeling

In [149]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import iqr, randint, uniform
import lightgbm as lgb
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import Imputer, StandardScaler, RobustScaler
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, train_test_split, cross_val_score
from sklearn.metrics import roc_auc_score
from skopt import gp_minimize
from skopt.plots import plot_convergence, plot_objective
import pickle
import gc

%load_ext autotime

gc.enable()
np.random.seed(235)

path = "/Users/dsaxton/home_credit_default/"

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 78.9 ms


#### Load the data

In [168]:
df = pd.read_csv(path + "train.csv")

time: 25.9 s


In [169]:
sk_id_curr = df.pop("SK_ID_CURR")
y = df.pop("TARGET")

time: 421 ms


#### Select features

In [170]:
clf = lgb.LGBMClassifier(n_estimators=1000, num_leaves=23, subsample=0.5)
clf.fit(df, y)

lgb_cols = df.columns[clf.feature_importances_ > 0]
len(lgb_cols)

465

time: 1min 42s


In [171]:
var_imp = pd.DataFrame({"Feature": df.columns, "Importance": clf.feature_importances_})[["Feature", "Importance"]].sort_values("Importance", ascending=False)

time: 4.53 ms


#### View important features

In [172]:
var_imp.head(20)

,Feature,Importance
31,EXT_SOURCE_2,347
32,EXT_SOURCE_3,344
366,AMT_CREDIT_DIV_AMT_ANNUITY,283
406,EXT_SOURCE_1_DIV_DAYS_BIRTH,247
394,DAYS_REGISTRATION_DIV_DAYS_LAST_PHONE_CHANGE,220
8,DAYS_BIRTH,206
408,AVG_AGG_SYNTHETIC_TARGET,194
358,TOTAL_AMT_CREDIT_SUM_DIV_SUM_DAYS_CREDIT_ENDDATE,175
9,DAYS_EMPLOYED,172
410,MAX_AGG_SYNTHETIC_TARGET,169


time: 7.23 ms


#### LightGBM CV

In [173]:
params = {"n_estimators": 20000, 
          "num_leaves": 500, 
          "min_data_in_leaf": 2000,
          "learning_rate": 0.005, 
          "bagging_fraction": 0.5, 
          "bagging_freq": 1, 
          "feature_fraction": 0.5, 
          "lambda_l2": 0.8}

lgb_data = lgb.Dataset(data=df, 
                       label=y)

cv_result = lgb.cv(params=params, 
                   train_set=lgb_data, 
                   nfold=5, 
                   metrics="auc", 
                   early_stopping_rounds=200, 
                   stratified=True, 
                   shuffle=True, 
                   verbose_eval=100, 
                   show_stdv=True, 
                   seed=2357)

cv_result = pd.DataFrame(cv_result)

/Users/dsaxton/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:390: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[100]	cv_agg's auc: 0.765106 + 0.00215563
[200]	cv_agg's auc: 0.769388 + 0.0019427
[300]	cv_agg's auc: 0.773212 + 0.00185377
[400]	cv_agg's auc: 0.776508 + 0.00183201
[500]	cv_agg's auc: 0.779312 + 0.00179058
[600]	cv_agg's auc: 0.781665 + 0.00177562
[700]	cv_agg's auc: 0.783554 + 0.00184411
[800]	cv_agg's auc: 0.785042 + 0.00195634
[900]	cv_agg's auc: 0.786237 + 0.00198023
[1000]	cv_agg's auc: 0.787221 + 0.00199014
[1100]	cv_agg's auc: 0.788066 + 0.0020144
[1200]	cv_agg's auc: 0.78878 + 0.00200994
[1300]	cv_agg's auc: 0.789397 + 0.00204267
[1400]	cv_agg's auc: 0.789943 + 0.0020524
[1500]	cv_agg's auc: 0.790448 + 0.00211626
[1600]	cv_agg's auc: 0.79078 + 0.00209317
[1700]	cv_agg's auc: 0.791083 + 0.0021335
[1800]	cv_agg's auc: 0.791353 + 0.00217204
[1900]	cv_agg's auc: 0.791559 + 0.00220188
[2000]	cv_agg's auc: 0.791777 + 0.00221606
[2100]	cv_agg's auc: 0.791896 + 0.00222837
[2200]	cv_agg's auc: 0.792032 + 0.00224156
[2300]	cv_agg's auc: 0.792122 + 0.00225751
[2400]	cv_agg's auc: 0.792

In [174]:
cv_result.tail()

,auc-mean,auc-stdv
2763,0.792345,0.002261
2764,0.792345,0.002262
2765,0.792342,0.002259
2766,0.792344,0.002259
2767,0.792346,0.002259


time: 7.73 ms


```
params = {"n_estimators": 20000, 
          "num_leaves": 500, 
          "min_data_in_leaf": 2000,
          "learning_rate": 0.003, 
          "bagging_fraction": 0.5, 
          "feature_fraction": 0.5, 
          "lambda_l2": 1}
```

#### Fit chosen model

CV mean:  0.792346

In [ ]:
lgb_model = lgb.LGBMClassifier(n_estimators=2767, 
                         num_leaves=500, 
                         min_data_in_leaf=1000,
                         learning_rate=0.005, 
                         bagging_fraction=0.5, 
                         feature_fraction=0.5, 
                         lambda_l2=0.8)

lgb_model.fit(df[lgb_cols], y)

#### Score test data and generate submission

In [ ]:
df = pd.read_csv(path + "test.csv")

In [ ]:
submission = pd.DataFrame({"SK_ID_CURR": df["SK_ID_CURR"], 
                           "TARGET": lgb_model.predict_proba(df[lgb_cols])[:,1]})

submission.head()

In [ ]:
submission.to_csv(path + "submission.csv", index=False)